In [104]:
import os
import numpy as np
from utils.image import read_reorient_nifti
import nibabel as nib
from utils.analysis import compute_dice
import json

In [105]:
res_dir = '/media/t722s/2.0 TB Hard Disk/lesions_experiments_20240729/'
data_dir = '/home/t722s/Desktop/Datasets/melanoma_HD'

In [152]:
models = ['sam', 'medsam', 'sammed2d']
splits = ['Tr', 'Ts']
zeros = {}
for model in models:
    with open('/media/t722s/2.0 TB Hard Disk/lesions_experiments_20240729/' + model + '_melanoma_HD/results.json', 'r') as f:
                results = json.load(f)
    zeros[model] = {
        os.path.join('labels' + split,filename): instances
        for split in ['Tr', 'Ts']
        for filename, instances in results[split].items()
        if any(score is None for score in instances.values()) 
    }

In [153]:
zeros

{'sam': {'labelsTr/0001996954_Follow-Up_2.nii.gz': {'1': 0.788235294117647,
   '2': 0.6836938580488229,
   '4': 0.6791666666666667,
   '5': 0.583941605839416,
   '7': 0.5907158836689038,
   '8': 0.7067750677506776,
   '9': 0.7794719226478245,
   '10': 0.6537732271520668,
   '11': 0.6959349593495935,
   '12': 0.7427055702917772,
   '13': 0.6492618917441225,
   '15': None,
   '16': 0.7968683043211378,
   '18': 0.7439791082309701,
   '19': 0.7791164658634538,
   '20': 0.6286644951140065,
   '25': 0.46334824640991146,
   '26': 0.5210638386706192,
   '27': 0.6024012769796655,
   '28': 0.6907216494845361,
   '29': 0.8441791482037573,
   '30': 0.5194616224081484,
   '31': 0.7690750860149768,
   '32': 0.5899714376388449,
   '33': 0.7696481812760883,
   '34': 0.6838323353293413,
   '35': 0.7496087636932708,
   '37': 0.7135231316725978,
   '40': 0.6504559270516718,
   '41': 0.6839062975357468,
   '42': 0.7560202043932809,
   '43': 0.6765982065374603,
   'all': 0.15593751891127985},
  'labelsTr/0

In [144]:
{0,1,2}.intersection({1,2,3})

{1, 2}

In [147]:
all_0s = set(zeros['sam'].keys())
# for model in models:
#     all_0s = all_0s.intersection(set(zeros[model].keys()))

In [148]:
all_0s

{'labelsTs/0002045557_Baseline.nii.gz'}

In [134]:

res_bad_all = {model:{split:{} for split in splits} for model in models}

for model in models:
    for split in splits:

        with open('/media/t722s/2.0 TB Hard Disk/lesions_experiments_20240729/' + model + '_melanoma_HD/results.json', 'r') as f:
            results = json.load(f)
            results = results[split]
        res_bad = {basename: {k:v for k,v in dice_scores.items() if v == 0}
                for basename, dice_scores in results.items()}
        res_bad = {os.path.join('labels' + split, k):v for k,v in res_bad.items() if len(v)>0}

        res_bad_all[model][split] = res_bad
    res_bad_all[model] = {basename:seg_res 
                          for split, splitdict in res_bad_all[model].items() 
                          for basename, seg_res in splitdict.items()}


In [135]:
res_bad_all

{'sam': {'labelsTs/0002045557_Baseline.nii.gz': {'5': 0.0}},
 'medsam': {'labelsTr/0002227163_Follow-Up_2.nii.gz': {'3': 0.0},
  'labelsTr/0000402877_Follow-Up_1.nii.gz': {'5': 0.0},
  'labelsTr/0000402877_Follow-Up_2.nii.gz': {'3': 0.0},
  'labelsTr/0001646391_Baseline.nii.gz': {'4': 0.0, '6': 0.0},
  'labelsTr/0002531979_Follow-Up_3.nii.gz': {'4': 0.0},
  'labelsTr/0002178452_Follow-Up_1.nii.gz': {'4': 0.0},
  'labelsTr/0001646391_Follow-Up_1.nii.gz': {'3': 0.0},
  'labelsTr/0001646391_Follow-Up_2.nii.gz': {'4': 0.0},
  'labelsTs/0002752644_Baseline.nii.gz': {'24': 0.0},
  'labelsTs/0002674005_Follow-Up_1.nii.gz': {'2': 0.0},
  'labelsTs/0002212009_Follow-Up_1.nii.gz': {'1': 0.0, 'all': 0.0},
  'labelsTs/0002677387_Follow-Up_2.nii.gz': {'10': 0.0, '12': 0.0},
  'labelsTs/0002764960_Follow-Up_2.nii.gz': {'1': 0.0,
   '2': 0.0,
   '4': 0.0,
   '5': 0.0},
  'labelsTs/0002627385_Follow-Up_1.nii.gz': {'21': 0.0, '22': 0.0, '26': 0.0},
  'labelsTs/0002955329_Baseline.nii.gz': {'6': 0.0},
 

In [136]:
sam_bad = res_bad_all['sam']
medsam_bad = res_bad_all['medsam']
sammed2d_bad = res_bad_all['sammed2d']

gt_instance_pairs = {}
for model in models:
    model_bad = res_bad_all['model']
    gt_instances_model = {}


{'labelsTs/0002045557_Baseline.nii.gz': {'5': 0.0}}

In [ ]:
split = 'Ts'
model = 'sam'